### Summary
- fasttext Features are added (can be skipped and pickled file used)
- Functions that run n multiple predictions for seperate models (SVM, NB, Logit) on the dataset.
- using different sets of features
- returning the mean and variance of accuracy over the n runs.
- results are exported as Excel Files

In [43]:
#importing libraries for NLP
import pandas as pd
import os as os
import json as js
import gzip
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

#Image processing material
import imageio
from PIL import Image
from tqdm import tqdm
from scipy.special import expit as sigmoid
from scipy.stats import multivariate_normal

#importing libraries for NLP
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

#Text Classification Material
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
import scipy as sp

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MAHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import gensim
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath
from gensim.models import FastText

### Dataframe
Use the pickled full-feature dataframes

In [2]:
with open ('df_07_08', 'rb') as fr:
    df = pickle.load(fr)

In [4]:
df.head()

,reviewerID,asin,reviewText,overall,good,Comedy,Art House & International,Drama,Action & Adventure,Horror,...,corpus,noun,adj,verb (simple),verb (simple past),verb (past participle),verb (gerund),adverb,preposition,sentiment
0,A19JYLHD94K94D,6304239327,Jan Svankajer's feature film follow up to his ...,5.0,1,0,1,1,0,0,...,jan svankaj featur film follow masterpiec alic...,0.589624,0.801433,-1.084828,-0.866063,-1.115746,0.020607,0.578095,0.141700,0.9904
1,A144W50UYZG4YX,630339406X,Big-1988100mins/ColourTom HanksElizabeth Perki...,4.0,1,0,0,0,0,0,...,big min colourtom hankselizabeth perkinsrobert...,0.923381,1.265269,-1.372903,-0.648728,0.800474,0.674204,-0.962428,0.004977,0.9934
2,A16QODENBJVUI1,B0007PALGG,Of all the films coming out this holiday movie...,4.0,1,0,0,1,0,0,...,film come holiday movi season kinsey near top ...,0.789278,0.328968,-0.437079,0.246639,-0.340824,0.149929,-0.029449,1.596602,0.9962
3,A3L2856DIMUXJY,B0001DCR0C,"As noted by a number of reviewers, this starte...",4.0,1,0,1,1,0,0,...,note number review start life play director ma...,0.210671,-0.553402,0.253301,0.445873,0.343758,0.523410,-1.190013,1.242949,0.9455
4,A1Z54EM24Y40LL,B00004WI5C,This is really a cute video. It's pretty long...,4.0,1,0,0,0,0,0,...,realli cute video pretti long though may hard ...,-1.755248,-0.521771,1.846551,-1.006187,-1.115746,-0.137556,1.073756,0.138101,0.9606


In [ ]:
def split_sentence(df):
    df["split"]=df["corpus"].apply(lambda x: x.split())
split_sentence(df)

In [ ]:
sentences = list(df.split)

In [ ]:
#takes about 10 min:
size=500
#Movie and TV Data
model_m = FastText(size=size, seed=1)  # instantiate
model_m.build_vocab(sentences=sentences)
model_m.train(sentences=sentences, total_examples=len(sentences), epochs=30)  # train

In [ ]:
def sent_vectorizer(sent, model):
    '''Uses a pretrained FastText model to create an avage over 
    a review.'''
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec) / numw

In [ ]:
def avg_sent(df,model,size):
    '''Takes a dataframe and adds features: Every dimension of
    a model becomes a column. Return new dataframe'''
    df["avg_sentence"]=df["split"].apply(lambda x: sent_vectorizer(x, model))
    length=df["avg_sentence"].apply(lambda x: len(x))
    df2=df[length==size]
    for i in range(size):
        if size==500:
            df2[str(i)] = df2["avg_sentence"].apply(lambda x : x[i])
        else:
            df2[str(1000+i)] = df2["avg_sentence"].apply(lambda x : x[i])
    return(df2)

In [ ]:
# Adding fasttext-500 features to df's
x_m=avg_sent(df, model_m, 500)

In [ ]:
# pickle full feature df's
with open ('df_07_08_fasttext', 'wb') as fw:
    pickle.dump(x_m, fw)

In [5]:
import pickle
with open ('df_07_08_fasttext', 'rb') as fr:
    x_m = pickle.load(fr)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

def prediction(df, fasttext=False, size=500,
               bow=False, top_words=500,
               personality=False,
               grammar=False, 
               sentiment=False,
               interaction = False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,
               model="logit",
               seed=1):
    '''This function return the accuracy of a prediction dependent on
    the set of features used.'''
    
    # Part I: Creating Feature Set
    if bow == True:
        # using n-grams as well
        cv = CountVectorizer(max_features = top_words, ngram_range=(1,2))
        XX = cv.fit_transform(df["corpus"]).toarray()
        # using relative word counts instead of absolute ones
        tfidf_transformer = TfidfTransformer()
        tfidf = tfidf_transformer.fit_transform(XX).toarray()
        # tfidf added
        X = tfidf


    if grammar == True:
        # adding Grammar features
        grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition']
        X=df[grammatical]

    if personality == True:
        # personality added
        big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
        X=df[big5]

  
    if interaction == True:
        # personality added
        interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
        'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
        'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
        'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']
                      
        X=df[interactions] 

    if personality==interaction==grammar==True:
        interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
        'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
        'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
        'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']        
        big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
        grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition']
        combined=interactions+big5+grammatical
        X=df[combined]

    if fasttext==True:
        if size==500:
            dimensions=list(range(size))
            embedding = [str(i) for i in dimensions]
        else:
            dimensions=list(range(1000,1000+size))
            embedding = [str(i) for i in dimensions]
            
        if personality==True:
            big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
            embedding = big5 + embedding

            
        if grammar==True:
            grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition']
            embedding = grammatical + embedding
    
        if interaction==True:
            interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
        'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
        'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
        'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']
            embedding = interactions + embedding
            
        X=df[embedding]
        
        print(X.shape)
   
    if sentiment == True:
        X=np.array(df["sentiment"]).reshape(-1, 1)
  
    if bow_plus_personality == True:
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['Extro', 'Agree', 'Neuro', 'Open', 'Consc']].values),
                          format='csr')
        X=X.toarray()
 
    if bow_plus_personality_plus_interactions == True:  
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['Extro_Comedy',
           'Extro_Art House & International', 'Extro_Drama',
           'Extro_Action & Adventure', 'Extro_Horror', 'Extro_Science Fiction',
           'Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International',
           'Agree_Drama', 'Agree_Action & Adventure', 'Agree_Horror',
           'Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',
           'Neuro_Art House & International', 'Neuro_Drama',
           'Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',
           'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International',
           'Open_Drama', 'Open_Action & Adventure', 'Open_Horror',
           'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy',
           'Consc_Art House & International', 'Consc_Drama',
           'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction',
           'Consc_Animation']].values),
                          format='csr')
        
    if bow_plus_personality_plus_grammar == True:
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['length', '!', '?', ',', '.', ' ','noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition','Extro', 'Agree', 'Neuro', 'Open', 'Consc']].values),
                          format='csr')
        
    if bow_plus_personality_plus_interactions_plus_grammar == True:
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['length', '!', '?', ',', '.', ' ','noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition','Extro', 'Agree', 'Neuro', 'Open', 'Consc', 'Extro_Comedy',
       'Extro_Art House & International', 'Extro_Drama',
       'Extro_Action & Adventure', 'Extro_Horror', 'Extro_Science Fiction',
       'Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International',
       'Agree_Drama', 'Agree_Action & Adventure', 'Agree_Horror',
       'Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',
       'Neuro_Art House & International', 'Neuro_Drama',
       'Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International',
       'Open_Drama', 'Open_Action & Adventure', 'Open_Horror',
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy',
       'Consc_Art House & International', 'Consc_Drama',
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction',
       'Consc_Animation']].values),
                          format='csr')
    
    # Part II: Running a logit classification on the corresponding feature set X
    y=df["good"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)
    if model=="logit":
        log = LogisticRegression(solver='liblinear')#regularization is applied by default
    elif model=="nb":
        log = GaussianNB()
        #X = X.todense()
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)
    elif model=="nn":
        log = MLPClassifier()
    elif model=="svm":
        log = SVC(gamma='auto')

    log.fit(X_train, y_train)
    accuracy=log.score(X_test,y_test)
    #y_pred = log.predict(X_test)
    #cm = confusion_matrix(y_test, y_pred)
    #print(cm)
    
    return(accuracy)

Add word types and sentiment later on

In [36]:
columns=["Mean", "SD"]
tested=["Bag-of-Words_500",
        "Bag-of-Words_1000",
        "fastText",
        "Personality",
        "Grammar",
        "Sentiment",
        "Interaction",
        "Personality+Grammar+Interaction",
        "BOW+Personality",
        "BOW+Personality+Interactions",
        "BOW+Personality+Interactions+Grammar",
        "BOW+Personality+Grammar",
        "fasttext+Personality",
        "fasttext+Personality+Interactions+Grammar",
        "fasttext+Personality+Interactions",
        "fasttext+Personality+Grammar"
       ]
result_df=pd.DataFrame(columns=columns, index=tested)
result_df

,Mean,SD
Bag-of-Words_500,NaN,NaN
Bag-of-Words_1000,NaN,NaN
fastText,NaN,NaN
Personality,NaN,NaN
Grammar,NaN,NaN
Sentiment,NaN,NaN
Interaction,NaN,NaN
Personality+Grammar+Interaction,NaN,NaN
BOW+Personality,NaN,NaN
BOW+Personality+Interactions,NaN,NaN


In [20]:
import statistics

# FULL Version: Use this for Logit 
def create_results(results,n, model):
    '''This functions shows the mean accuracy over n samples.'''
    df=x_m
    
    # 1 BoW
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, bow=True,top_words=500,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Bag-of-Words_500","Mean"] = mean
    results.loc["Bag-of-Words_500","SD"] = sd

    # 2 Tfidf-1000
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, bow=True,top_words=1000,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Bag-of-Words_1000","Mean"] = mean
    results.loc["Bag-of-Words_1000","SD"] = sd

    # 3 fasttext-500
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, fasttext=True,size=500,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["fastText","Mean"] = mean
    results.loc["fastText","SD"] = sd

    # 4 personality
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, personality=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Personality","Mean"] = mean
    results.loc["Personality","SD"] = sd

 
    # 5 Personality genre interaction
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, interaction=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Interaction","Mean"] = mean
    results.loc["Interaction","SD"] = sd

    # 6 Grammar
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, grammar=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Grammar","Mean"] = mean
    results.loc["Grammar","SD"] = sd
    
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, personality=True,interaction=True,grammar=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Personality+Grammar+Interaction","Mean"] = mean
    results.loc["Personality+Grammar+Interaction","SD"] = sd    
    
    
    # 7 BOW_plus_personality
       
    
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality = True ,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["BOW+Personality","Mean"] = mean
    results.loc["BOW+Personality","SD"] = sd
    
     # 8 BOW_plus_personality_plus_interactions
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality_plus_interactions = True ,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "BOW+Personality+Interactions","Mean"] = mean
    results.loc[ "BOW+Personality+Interactions","SD"] = sd
    
    # 9 BOW_plus_personality_plus_interactions_plus_grammar
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality_plus_grammar = True ,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "BOW+Personality+Grammar","Mean"] = mean
    results.loc[ "BOW+Personality+Grammar","SD"] = sd
    
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality_plus_interactions_plus_grammar = True ,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "BOW+Personality+Interactions+Grammar","Mean"] = mean
    results.loc[ "BOW+Personality+Interactions+Grammar","SD"] = sd
    
     10 VADER sentiment
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, sentiment = True ,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "Sentiment","Mean"] = mean
    results.loc[ "Sentiment","SD"] = sd
    
    
    # 11 fasttext_plus_personality
       
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True , personality=True, size=500,
               bow=False, top_words=500,
               grammar=False, 
               sentiment=False,
               interaction = False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["fasttext+Personality","Mean"] = mean
    results.loc["fasttext+Personality","SD"] = sd
    
     # 12 BOW_plus_personality_plus_interactions
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True ,personality=True, interaction=True,size=500,
               bow=False, top_words=500,
               grammar=False, 
               sentiment=False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "fasttext+Personality+Interactions","Mean"] = mean
    results.loc[ "fasttext+Personality+Interactions","SD"] = sd
    
    # 9 BOW_plus_personality_plus_interactions_plus_grammar
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True ,personality=True,grammar=True, size=500,
               bow=False, top_words=500,
               sentiment=False,
               interaction = False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "fasttext+Personality+Grammar","Mean"] = mean
    results.loc[ "fasttext+Personality+Grammar","SD"] = sd
    
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True ,personality=True,interaction=True, size=500,
               bow=False, top_words=500,
               grammar=True, 
               sentiment=False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "fasttext+Personality+Interactions+Grammar","Mean"] = mean
    results.loc[ "fasttext+Personality+Interactions+Grammar","SD"] = sd
        
    
    return(results)

In [37]:
import statistics

#Smaller Version: For SVM and NB
def create_results2(results,n, model):
    '''This functions shows the mean accuracy over n samples.'''
    df=x_m
    
    # 1 BoW
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, bow=True,top_words=500,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Bag-of-Words_500","Mean"] = mean
    results.loc["Bag-of-Words_500","SD"] = sd

    # 2 Tfidf-1000
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, bow=True,top_words=1000,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Bag-of-Words_1000","Mean"] = mean
    results.loc["Bag-of-Words_1000","SD"] = sd

    # 3 fasttext-500
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, fasttext=True,size=500,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["fastText","Mean"] = mean
    results.loc["fastText","SD"] = sd

    # 4 personality
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, personality=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Personality","Mean"] = mean
    results.loc["Personality","SD"] = sd

 
    # 5 Personality genre interaction
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, interaction=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Interaction","Mean"] = mean
    results.loc["Interaction","SD"] = sd

    # 6 Grammar
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, grammar=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Grammar","Mean"] = mean
    results.loc["Grammar","SD"] = sd
    
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, personality=True,interaction=True,grammar=True,seed=seed,model=model)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Personality+Grammar+Interaction","Mean"] = mean
    results.loc["Personality+Grammar+Interaction","SD"] = sd    
    
    
    return(results)



In [ ]:
resu=create_results(result_df,2)

In [ ]:
resu25 = create_results(result_df,10)
resu25

In [ ]:
resu25.to_excel("results_07_08.xlsx")

In [ ]:
resuNN=create_results2(result_df,2,model="nn")
resuNN

In [ ]:
resuNN

In [38]:
resuNB=create_results2(result_df,2,model="nb")
resuNB
#10:12 -

(18808, 500)
(18808, 500)


,Mean,SD
Bag-of-Words_500,0.773525,0.00526289
Bag-of-Words_1000,0.789607,0.00394717
fastText,0.645534,0.000563881
Personality,0.587719,0.0105258
Grammar,0.615497,0.00169164
Sentiment,NaN,NaN
Interaction,0.56446,0.0125933
Personality+Grammar+Interaction,0.576954,0.00921006
BOW+Personality,NaN,NaN
BOW+Personality+Interactions,NaN,NaN


In [31]:
resuNB

,Mean,SD
Bag-of-Words_500,0.773525,0.00526289
Bag-of-Words_1000,0.789607,0.00394717
fastText,0.645534,0.000563881
Personality,0.587719,0.0105258
Grammar,0.615497,0.00169164
Sentiment,NaN,NaN
Interaction,0.56446,0.0125933
Personality+Grammar+Interaction,0.576954,0.00921006
BOW+Personality,NaN,NaN
BOW+Personality+Interactions,NaN,NaN


In [39]:
resuNB.to_excel("results_10_08_NB.xlsx")

In [42]:
#10;20
from sklearn.svm import SVC
resuSVM=create_results2(result_df,10,model="svm")
resuSVM

,Mean,SD
Bag-of-Words_500,0.773525,0.00526289
Bag-of-Words_1000,0.789607,0.00394717
fastText,0.645534,0.000563881
Personality,0.587719,0.0105258
Grammar,0.615497,0.00169164
Sentiment,NaN,NaN
Interaction,0.56446,0.0125933
Personality+Grammar+Interaction,0.685646,0.00544357
BOW+Personality,NaN,NaN
BOW+Personality+Interactions,NaN,NaN


In [27]:
resuSVM.to_excel("results_10_08_SVM.xlsx")